In [1]:
import os

os.add_dll_directory(
    "C:\\Users\\ospin\\miniconda3\\envs\\xhydro-dev\\julia_env\\pyjuliapkg\\install\\bin"
)

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

# Basic imports
try:
    from xhydro.extreme_value_analysis.parameterestimation import *

    has_julia = True
except ImportError:
    print(
        "This notebook requires the `julia` programming language and the `juliacall` Python package to be installed."
    )
    has_julia = False

import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pooch

# warnings.simplefilter("always", UserWarning)

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


In [2]:
GITHUB_URL = "https://github.com/hydrologie/xhydro-testdata"
BRANCH_OR_COMMIT_HASH = "main"

data = pooch.retrieve(
    url=f"{GITHUB_URL}/raw/{BRANCH_OR_COMMIT_HASH}/data/extreme_value_analysis/NOAA_GFDL_ESM4.zip",
    known_hash="md5:6c0f51c59add2f37b0e4af169b7eb7f6",
    processor=pooch.Unzip(),
)

df = pd.read_csv(data[0])
ds = df.to_xarray()
ds = ds.set_coords(["time", "station_num", "station_name"])
ds = ds.set_index(index=["station_num", "time"])
ds = ds.unstack("index")
ds

<xarray.Dataset> Size: 455kB
Dimensions:          (station_num: 97, time: 146)
Coordinates:
  * station_num      (station_num) int64 776B 1001 1004 1008 ... 15403 15414
  * time             (time) object 1kB '1955-01-01' ... '2100-01-01'
    station_name     (station_num, time) object 113kB 'Dozois' ... 'Romaine 4...
Data variables:
    qmax_yearly      (station_num, time) float64 113kB 424.0 805.1 ... 711.7
    total_precip     (station_num, time) float64 113kB 983.6 ... 1.065e+03
    max_temp_yearly  (station_num, time) float64 113kB 8.0 8.1 8.1 ... 4.4 4.6

In [4]:
chosen_method = "BAYES"

# 1. GEV

1.1.1. Parameter stationary 

In [ ]:
params = fit(
    ds,
    dist="genextreme",
    method=chosen_method,
    vars=["total_precip"],
    confidence_level=0.95,
)
params

Progress: 100%|█████████████████████████████████████████| Time: 0:00:12
Progress: 100%|█████████████████████████████████████████| Time: 0:00:12
Progress: 100%|█████████████████████████████████████████| Time: 0:00:11
Progress: 100%|█████████████████████████████████████████| Time: 0:00:13
Progress: 100%|█████████████████████████████████████████| Time: 0:00:12
Progress: 100%|█████████████████████████████████████████| Time: 0:00:13
Progress: 100%|█████████████████████████████████████████| Time: 0:00:11
Progress: 100%|█████████████████████████████████████████| Time: 0:00:12
Progress: 100%|█████████████████████████████████████████| Time: 0:00:12
Progress: 100%|█████████████████████████████████████████| Time: 0:00:11
Progress: 100%|█████████████████████████████████████████| Time: 0:00:11
Progress: 100%|█████████████████████████████████████████| Time: 0:00:11
Progress: 100%|█████████████████████████████████████████| Time: 0:00:09
Progress: 100%|█████████████████████████████████████████| Time: 

1.1.2. Parameter non-stationary

In [5]:
params = fit(
    ds,
    dist="genextreme",
    method=chosen_method,
    vars=["total_precip"],
    locationcov=["max_temp_yearly"],
    distributed=True,
)
params

ValueError: Probability weighted moment parameter estimation cannot have covariates ['max_temp_yearly']

1.2.1. Return level stationary

In [3]:
return_level_quantiles_stat = return_level(
    ds,
    dist="genextreme",
    method=chosen_method,
    vars=["total_precip"],
    return_period=100,
)
return_level_quantiles_stat

NameError: name 'chosen_method' is not defined

1.2.2. Return level non-stationary

In [ ]:
return_level_quantiles_nstat = return_level(
    ds,
    dist="genextreme",
    method=chosen_method,
    vars=["total_precip"],
    return_period=100,
    locationcov=["max_temp_yearly"],
    distributed=True,
)
return_level_quantiles_nstat

Progress: 100%|█████████████████████████████████████████| Time: 0:00:52
